# Understanding Bateman Equations in Nuclear Depletion

## Overview
The Bateman equations describe how nuclear material compositions change over time due to:
- Radioactive decay
- Neutron-induced reactions
- Fission product generation

We'll explore how these equations work and implement them in Python.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.integrate import odeint

## 1. Basic Decay Chain

The simplest form of the Bateman equations describes radioactive decay:

$\frac{dN_i}{dt} = \sum_{j} \lambda_{ji}N_j - \lambda_iN_i$

where:
- $N_i$ is the number density of nuclide i
- $\lambda_i$ is the decay constant of nuclide i
- $\lambda_{ji}$ is the decay constant from nuclide j to i

In [ ]:
def decay_chain(N0, lambda_vals, t):
    """Simple decay chain solver"""
    def dN_dt(N, t, lambda_vals):
        return [-lambda_vals['I135']*N[0],
                lambda_vals['I135']*N[0] - lambda_vals['Xe135']*N[1]]
    
    t_span = np.linspace(0, t, 100)
    solution = odeint(dN_dt, N0, t_span, args=(lambda_vals,))
    return t_span, solution

# Example: I-135 → Xe-135 decay
N0 = [1.0, 0.0]  # Initial concentrations
lambda_I135 = np.log(2)/(6.57*3600)  # I-135 half-life: 6.57 hours
lambda_Xe135 = np.log(2)/(9.14*3600)  # Xe-135 half-life: 9.14 hours
lambda_vals = {'I135': lambda_I135, 'Xe135': lambda_Xe135}

t_span, solution = decay_chain(N0, lambda_vals, 24*3600)  # 24 hours

plt.figure(figsize=(10, 6))
plt.plot(t_span/3600, solution[:, 0], label='I-135')
plt.plot(t_span/3600, solution[:, 1], label='Xe-135')
plt.xlabel('Time (hours)')
plt.ylabel('Relative concentration')
plt.title('I-135 → Xe-135 Decay Chain')
plt.grid(True)
plt.legend()
plt.show()

## 2. Adding Neutron Reactions

In a reactor, we must also consider neutron-induced reactions:

$\frac{dN_i}{dt} = \sum_{j} \lambda_{ji}N_j + \sum_{j} \sigma_{ji}\phi N_j - (\lambda_i + \sigma_i\phi)N_i$

where:
- $\phi$ is the neutron flux
- $\sigma_i$ is the total reaction cross section
- $\sigma_{ji}$ is the production cross section from j to i

In [ ]:
def depletion_matrix(phi, sigma_c, sigma_f, lambda_vals, yields):
    """Create depletion matrix for U235 → Xe135 chain"""
    A = np.zeros((3, 3))
    
    # U-235 disappearance
    A[0,0] = -(sigma_c['U235'] + sigma_f['U235'])*phi
    
    # I-135 production and decay
    A[1,0] = yields['I135']*sigma_f['U235']*phi
    A[1,1] = -lambda_vals['I135']
    
    # Xe-135 production and loss
    A[2,1] = lambda_vals['I135']
    A[2,0] = yields['Xe135']*sigma_f['U235']*phi
    A[2,2] = -(lambda_vals['Xe135'] + sigma_c['Xe135']*phi)
    
    return A

# Example parameters
phi = 1e14  # n/cm²/s
sigma_c = {'U235': 8.7e-24, 'Xe135': 2.6e-18}  # cm² (U-235, Xe-135)
sigma_f = {'U235': 5.8e-22, 'Xe135': 0.0}  # cm²
yields = {'I135': 0.063, 'Xe135': 0.003}  # I-135, Xe-135 direct


A = depletion_matrix(phi, sigma_c, sigma_f, lambda_vals, yields)

# Initial conditions (1kg U-235)
N0 = np.array([2.56e21, 0.0, 0.0])
times = np.linspace(0, 48*3600, 100)

# Solve using matrix exponential
solutions = [np.dot(expm(A*t), N0) for t in times]
solutions = np.array(solutions)

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(times/3600, solutions[:,0]/N0[0], label='U-235')
plt.xlabel('Time (hours)')
plt.ylabel('Relative concentration')
plt.title('U-235 Depletion')
plt.grid(True)
plt.legend()

plt.subplot(122)
plt.plot(times/3600, solutions[:,1], label='I-135')
plt.plot(times/3600, solutions[:,2], label='Xe-135')
plt.xlabel('Time (hours)')
plt.ylabel('Atoms/cm³')
plt.title('Fission Product Buildup')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## 3. Importance of Fission Product Yields

Fission products can be produced:
1. Directly from fission (direct yield)
2. Through decay of parent nuclides (cumulative yield)

For Xe-135:
- Direct yield: ~0.3%
- Cumulative yield: ~6.6% (mostly through I-135)

In [ ]:
def compare_yields(phi):
    """Compare Xe-135 buildup with/without I-135"""
    yields_direct = {'I135': 0.00, 'Xe135': 0.003}  # All yield direct to Xe-135
    yields_chain = {'I135': 0.063, 'Xe135': 0.003}  # Realistic yields
    
    A_direct = depletion_matrix(phi, sigma_c, sigma_f, lambda_vals, yields_direct)
    A_chain = depletion_matrix(phi, sigma_c, sigma_f, lambda_vals, yields_chain)
    print(A_direct)
    solutions_direct = [np.dot(expm(A_direct*t), N0) for t in times]
    solutions_chain = [np.dot(expm(A_chain*t), N0) for t in times]
    
    return np.array(solutions_direct), np.array(solutions_chain)

s_direct, s_chain = compare_yields(phi)

plt.figure(figsize=(10, 6))
plt.plot(times/3600, s_direct[:,2], '--', label='Direct yield only')
plt.plot(times/3600, s_chain[:,2], label='Full decay chain')
plt.xlabel('Time (hours)')
plt.ylabel('Xe-135 concentration (atoms/cm³)')
plt.title('Impact of Decay Chain on Xe-135 Buildup')
plt.grid(True)
plt.legend()
plt.show()

## 4. Flux Effects

The neutron flux affects:
1. Rate of fission product generation
2. Burnup rate of fissile material
3. Transmutation rates

Let's examine how different flux levels impact the system:

In [ ]:
flux_levels = [1e13, 1e14, 1e15]
plt.figure(figsize=(12, 4))

for phi in flux_levels:
    A = depletion_matrix(phi, sigma_c, sigma_f, lambda_vals, yields)
    solutions = [np.dot(expm(A*t), N0) for t in times]
    solutions = np.array(solutions)
    
    plt.plot(times/3600, solutions[:,2], 
             label=f'ϕ = {phi:0.1e} n/cm²/s')

plt.xlabel('Time (hours)')
plt.ylabel('Xe-135 concentration')
plt.title('Flux Impact on Xe-135 Buildup')
plt.grid(True)
plt.legend()
plt.show()

## 5. Matrix Exponential Method

The general solution to the Bateman equations is:

$\vec{N}(t) = e^{At}\vec{N}(0)$

where A is the transition matrix containing all rates.

Advantages:
- Exact solution
- Handles stiff systems well
- No time step restrictions

Disadvantages:
- Computationally expensive for large systems
- Memory intensive
- Requires constant coefficients

In [ ]:
def compare_methods(N0, A, times):
    """Compare matrix exponential vs numerical integration"""
    # Matrix exponential solution
    solutions_matrix = [np.dot(expm(A*t), N0) for t in times]
    
    # Numerical integration
    def dN_dt(N, t, A):
        return np.dot(A, N)
    
    solutions_ode = odeint(dN_dt, N0, times, args=(A,))
    
    return np.array(solutions_matrix), solutions_ode

s_matrix, s_ode = compare_methods(N0, A, times)

plt.figure(figsize=(10, 6))
plt.plot(times/3600, s_matrix[:,2], 'k--', label='Matrix exponential')
plt.plot(times/3600, s_ode[:,2], 'r:', label='Numerical integration')
plt.xlabel('Time (hours)')
plt.ylabel('Xe-135 concentration')
plt.title('Comparison of Solution Methods')
plt.grid(True)
plt.legend()
plt.show()

# Show relative difference
rel_diff = np.abs(s_matrix[1:,2] - s_ode[1:,2])/s_matrix[1:,2]
print(f'Maximum relative difference: {rel_diff.max():0.2e}')

## 6. Practical Considerations

In real reactor calculations:
1. Much larger number of nuclides (>1000)
2. Flux varies with position and time
3. Cross sections change with burnup
4. Temperature effects

Solutions:
- Use predictor-corrector methods
- Split time steps into substeps
- Apply various acceleration techniques

### Predictor-Corrector Method Explained

The predictor-corrector method is an iterative approach used to solve differential equations, like the Bateman equations, with improved accuracy compared to single-step methods.

### 1. Predictor Step

The predictor step uses the current solution to estimate the solution at the next time point.

- **Depletion Matrix Calculation:**
    - Calculates the depletion matrix ($A$) based on the current state $N$ using $A\_func(N)$. This function takes the current number densities of the nuclides and calculates the macroscopic cross-sections, decay constants, and other parameters needed to construct the matrix.
- **State Prediction:**
    - Applies the matrix exponential method:
        $$N_{\text{pred}} = e^{A \cdot dt} \cdot N$$
    where:
    - $N_{\text{pred}}$ is the predicted state
    - $A$ is the depletion matrix
    - $dt$ is the time step
    - $N$ is the current state

### 2. Corrector Step

The corrector step refines the estimate using the predicted state.

- **Updated Matrix Calculation:**
    - Calculates a new depletion matrix $A_{\text{new}}$ based on the *predicted* state $N_{\text{pred}}$ using $A\_func(N_{\text{pred}})$.  This is the key difference from a simple forward Euler method. Because the nuclide concentrations have changed (as estimated by the predictor step), the reaction rates, and therefore the depletion matrix, are updated to reflect the new composition. For example, if the cross-sections are burnup-dependent, the burnup is estimated using $N_{\text{pred}}$ to calculate the updated cross-sections, which are then used to construct $A_{\text{new}}$.
- **State Correction:**
    $$N_{\text{corrected}} = e^{A_{\text{new}} \cdot dt} \cdot N$$

In [ ]:
def predictor_corrector(N0, A_func, times):
    """Simple predictor-corrector method"""
    N = N0.copy()
    results = [N0]
    
    for i in range(1, len(times)):
        dt = times[i] - times[i-1]
        
        # Predictor step
        A = A_func(N)
        N_pred = np.dot(expm(A*dt), N)
        
        # Corrector step
        A = A_func(N_pred)
        N = np.dot(expm(A*dt), N)
        
        results.append(N)
    
    return np.array(results)

# Let us assume a simple burnup-dependent cross section
def A_burnup_dependent(N):
    burnup = (N0[0] - N[0])/N0[0]  # Simple burnup estimate
    sigma_c_bu = {'U235': sigma_c['U235'] * (1 + 0.1*burnup), 'Xe135': sigma_c['Xe135'] * (1-0.2*burnup)}
    return depletion_matrix(phi, sigma_c_bu, sigma_f, lambda_vals, yields)

solutions_pc = predictor_corrector(N0, A_burnup_dependent, times)

plt.figure(figsize=(10, 6))
plt.plot(times/3600, s_matrix[:,2], 'k--', label='Constant XS')
plt.plot(times/3600, solutions_pc[:,2], 'r-', label='Burnup-dependent XS')
plt.xlabel('Time (hours)')
plt.ylabel('Xe-135 concentration')
plt.title('Effect of Burnup-Dependent Cross Sections')
plt.grid(True)
plt.legend()
plt.show()

## 7. Summary

Key points:
1. Bateman equations couple decay and neutron-induced reactions
2. Matrix exponential provides exact solution for constant coefficients
3. Decay chains and fission yields are crucial for accurate results
4. Flux level strongly impacts isotope evolution
5. Real calculations require sophisticated numerical methods